In [6]:
!pip install --upgrade pip
!pip install langchain langchain-community chromadb transformers sentence-transformers torch accelerate


Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
  Using cached distro-1.9.0-py3-none-any.whl.metadata (6.8 kB)
  Using cached pyproject_hooks-1.2.0-py3-none-any.whl.metadata (1.3 kB)
  Using cached markdown_it_py-4.0.0-py3-none-any.whl.metadata (7.3 kB)
  Using cached mdurl-0.1.2-py3-none-any.whl.metadata (1.6 kB)
   ---------------------------------------- 0.0/21.4 MB ? eta -:--:--
   ---------------------------------------- 0.3/21.4 MB ? eta -:--:--
   - -------------------------------------- 0.8/21.4 MB 2.5 MB/s eta 0:00:09
   - -------------------------------------- 1.0/21.4 MB 1.9 MB/s eta 0:00:11
   -- ------------------------------------- 1.3/21.4 MB 1.7 MB/s eta 0:00:12
   --- ------------------------------------ 1.8/21.4 MB 1.9 MB/s eta 0:00:11
   ---- ----------------------------------- 2.4/21.4 MB 2.2 MB/s eta 0:00:09
   ----- ---------------------------------- 2.9/21.

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
mysql-connector-python 8.1.0 requires protobuf<=4.21.12,>=4.21.1, but you have protobuf 6.33.5 which is incompatible.


In [10]:
!pip install protobuf==4.21.12 --force-reinstall


Defaulting to user installation because normal site-packages is not writeable
  Using cached protobuf-4.21.12-cp310-abi3-win_amd64.whl.metadata (541 bytes)
Using cached protobuf-4.21.12-cp310-abi3-win_amd64.whl (527 kB)
  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.5
    Uninstalling protobuf-6.33.5:
      Successfully uninstalled protobuf-6.33.5


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
opentelemetry-proto 1.39.1 requires protobuf<7.0,>=5.0, but you have protobuf 4.21.12 which is incompatible.


In [23]:
!pip install -q langchain langchain-community sentence-transformers chromedb transformers torch accelerate bitsandbytes 2>&1 | grep -v "dependency conflicts\|incompatible\|ERROR: pip's dependency" || true
print("\nInstallation complete!\n")


Installation complete!



'grep' is not recognized as an internal or external command,
operable program or batch file.


In [9]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import Chroma
from langchain_community.llms import HuggingFacePipeline

from langchain_core.prompts import PromptTemplate
from langchain_core.documents import Document
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline


In [10]:
print("\nStep 1: Creating knowledge base...")

documents = [
    Document(
        page_content="The Company Insurance Policy defines the insurance coverage provided by the organization to protect employees, company assets, and business operations from unforeseen risks and liabilities.",
        metadata={"source": "policy_intro"}
    ),
    Document(
        page_content="The scope of this insurance policy applies to all permanent and contractual employees, company-owned assets, and official business operations conducted on behalf of the organization.",
        metadata={"source": "policy_scope"}
    ),
    Document(
        page_content="The company provides employee insurance coverage including health insurance for hospitalization and medical treatment, accidental insurance for injury or disability, and life insurance where applicable, subject to insurer terms.",
        metadata={"source": "employee_insurance"}
    ),
    Document(
        page_content="Asset and infrastructure insurance covers company property against fire, natural disasters, theft, accidental damage, and equipment failure, including office premises, hardware, and servers.",
        metadata={"source": "asset_insurance"}
    ),
    Document(
        page_content="Business liability insurance protects the company against legal claims, third-party liabilities, professional negligence, and data security incidents where applicable.",
        metadata={"source": "liability_insurance"}
    ),
    Document(
        page_content="Insurance policies are administered by the Human Resources or Administration department. Policy details, coverage limits, and claim procedures are governed by the insurer and communicated to employees as required.",
        metadata={"source": "policy_administration"}
    ),
    Document(
        page_content="In the event of an incident, employees must notify the HR department promptly and submit required documentation within the insurer’s specified timeline for claim processing.",
        metadata={"source": "claims_procedure"}
    ),
    Document(
        page_content="Employees are responsible for providing accurate information, using insurance benefits ethically, and reporting claims honestly. Misuse of insurance benefits may result in disciplinary action.",
        metadata={"source": "employee_responsibility"}
    ),
    Document(
        page_content="All insurance-related personal and organizational data is treated as confidential, stored securely, and used only for policy administration and legal compliance.",
        metadata={"source": "data_privacy"}
    ),
    Document(
        page_content="The company reserves the right to review, amend, suspend, or terminate insurance coverage based on regulatory changes, insurer terms, or organizational requirements.",
        metadata={"source": "policy_revision"}
    ),
]



Step 1: Creating knowledge base...


In [11]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=50
)
splits = text_splitter.split_documents(documents)
print(f"Created {len(splits)} document chunks")

Created 10 document chunks


In [12]:
# Step 5: Create embeddings
print("\nStep 2: Loading embedding model...")
embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2"
)
print("Embedding model loaded")


Step 2: Loading embedding model...


C:\Users\David\AppData\Local\Temp\ipykernel_30980\512036564.py:3: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the `langchain-huggingface package and should be used instead. To use it run `pip install -U `langchain-huggingface` and import as `from `langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(


Embedding model loaded


In [13]:
#Step 6: Create vector store
print("\n Step 3: Creating vector database...")
vectorstore = Chroma.from_documents(
    documents=splits,
    embedding=embeddings,
    collection_name="rag_demo"
)
print(" Vector database created")


 Step 3: Creating vector database...
 Vector database created


In [22]:
# Step 7: Set up retriever
retriever = vectorstore.as_retriever(
    search_kwargs={"k": 2}  # Retrieve top 2 most relevant chunks
)

In [23]:
# Step 8: Load LLM (using a small model that works in Colab)
print("\n Step 4: Loading Language Model...")
print("   (Using FLAN-T5 - large model)")

model_name = "google/flan-t5-large"
tokenizer = AutoTokenizer.from_pretrained(model_name)



 Step 4: Loading Language Model...
   (Using FLAN-T5 - large model)


In [24]:
import torch
# Create pipeline directly (simpler and works better)
pipe = pipeline(
    "text2text-generation",
    model=model_name,
    tokenizer=tokenizer,
    max_new_tokens=256,  # Increased from max_length for better answers
    device=0 if torch.cuda.is_available() else -1,  # Use GPU if available
    model_kwargs={
        "temperature": 0.7,
        "do_sample": True,
        "top_p": 0.9,
    }
)

# Wrap in LangChain
llm = HuggingFacePipeline(pipeline=pipe)
print(" Language model loaded")

Device set to use cpu


 Language model loaded


In [25]:
# Step 9: Create custom prompt template
template = """Use the following pieces of context to answer the question at the end.
If you don't know the answer, just say that you don't know, don't try to make up an answer.
Keep the answer concise and relevant.

Context: {context}

Question: {question}

Answer:"""

QA_PROMPT = PromptTemplate(
    template=template,
    input_variables=["context", "question"]
)

In [26]:
# Step 10: Create RAG chain (NEW LangChain – LCEL)
print("\n Building RAG chain...")

from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

# qa_chain replaces RetrievalQA
qa_chain = (
    {
        "context": retriever,
        "question": RunnablePassthrough()
    }
    | QA_PROMPT
    | llm
    | StrOutputParser()
)

print(" RAG chain ready!")



 Building RAG chain...
 RAG chain ready!


In [27]:
# Step 11: Test the complete RAG system
print("\n" + "="*60)
print(" COMPANY INSURANCE POLICY – RAG DEMONSTRATION")
print("="*60)

def ask_question(query):
    print(f"\n Question: {query}")
    print("\n Step 1: Retrieving relevant documents...")

    # Explicit retrieval (LCEL does not hide this)
    docs = retriever.invoke(query)

    print(f"Retrieved {len(docs)} documents:")
    for i, doc in enumerate(docs, 1):
        print(f"\n   {i}. {doc.page_content[:150]}...")
        print(f"      Source: {doc.metadata['source']}")

    print("\n Step 2: Generating answer using LLM...")

    # LCEL invocation
    answer = qa_chain.invoke(query)

    print(f"\n Answer: {answer}")
    print("\n" + "-"*60)

# Run insurance-related demo queries
print("\n Running Insurance Policy Demo Queries...\n")

ask_question("What types of insurance coverage are provided to employees?")

ask_question("What assets are covered under the company insurance policy?")

ask_question("How should an employee file an insurance claim?")

ask_question("Who administers the company insurance policy?")

ask_question("How does the company protect insurance-related personal data?")



 COMPANY INSURANCE POLICY – RAG DEMONSTRATION

 Running Insurance Policy Demo Queries...


 Question: What types of insurance coverage are provided to employees?

 Step 1: Retrieving relevant documents...
Retrieved 2 documents:

   1. The company provides employee insurance coverage including health insurance for hospitalization and medical treatment, accidental insurance for injury...
      Source: employee_insurance

   2. The Company Insurance Policy defines the insurance coverage provided by the organization to protect employees, company assets, and business operations...
      Source: policy_intro

 Step 2: Generating answer using LLM...

 Answer: health insurance for hospitalization and medical treatment, accidental insurance for injury or disability, and life insurance

------------------------------------------------------------

 Question: What assets are covered under the company insurance policy?

 Step 1: Retrieving relevant documents...
Retrieved 2 documents:

   1. The C